In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/ReinforcementLearningAidedPIDControllerTuner.git -b work20200604

# # To update the work environment
# %cd /content/drive/My Drive/ReinforcementLearningAidedPIDControllerTuner
# !git pull

%cd /content/drive/My Drive/ReinforcementLearningAidedPIDControllerTuner/works/work20200604

In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))
sys.path.append(os.path.abspath("../../snippet"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import  ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

# 10. Misc

## 12. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## 14. a function to load gain and bias from the trains over a training:

In [ ]:
def getGainAndBiasOverTrain(buildOrderIdGiven, timeSimulationGiven):

    concStore = ConcStore()
    agentFactory = ConcAgentFactory()

    y = np.random.randn(1, ConcEnvironment.nPv).astype(np.float32)
    anObservationSequence = ObservationSequence()
    anObservationSequence.add(ConcObservation(y))

    cur.execute('''
    Select
        count(*)
        From TrainLog
        Where buildOrderId = ?
        ''', (buildOrderIdGiven,))
    assert cur.fetchone()[0] > 0, "NO DATA WITH THE GIVEN BUILD ID"

    cur.execute('''
    Select 
        trainId
        , timeSimulation
        , timestamp
        From TrainLog
        Where buildOrderId = ?
        And timeSimulation = ?
        Order By timeSimulation
        ''', (buildOrderIdGiven, timeSimulationGiven, ))


    (trainIdStr, timeSimulation, timestamp) = cur.fetchone()

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))

    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())
    agent(anObservationSequence)
    
    param = agent.getParameters() # param["gain"], param["bias"], param["sd"]

    return param, storeField.getBuildOrder().fixPolicySd, storeField.getBuildOrder().fixPolicyScale

# 20. Observe the trace of gain and bias over a traing

## 22. Choose a build order

In [ ]:
cur.execute('''
Select
    distinct buildOrderId
    From TrainLog
    Where buildOrderId like "casestudy1a %"
    Order by timestamp
    ''')

buildOrderIdsToExport = [elm for elm, in cur.fetchall()]

## 24. Load the parameters => SKIPPED

## 26. Plot the trace of Gain and Bias along with the simulation time => SKIPPED

## 28. Export the dataset of gains

In [ ]:
tbl = []

for buildOrderIdGiven in buildOrderIdsToExport:    
    cur.execute('''
Select
    distinct TimeSimulation
    From TrainLog
    Where buildOrderId = ?
    ''', (buildOrderIdGiven,))

    for timeSimulationGiven, in cur.fetchall():
        param, fixPolicySd, fixPolicyScale = getGainAndBiasOverTrain(buildOrderIdGiven, timeSimulationGiven)
        tbl.append((buildOrderIdGiven, timeSimulationGiven, param["gain"].squeeze(),param["bias"].squeeze(), param["sd"].squeeze(), 
                "fixPolicyScale = %d" % fixPolicyScale,
                "fixPolicySd = %d" % fixPolicySd))

In [ ]:
with open("cs1a_param.csv", "w") as fp:
    fp.write("buildOrderId,timeSimulation,gain,bias,sd,fixPolicySd,fixPolicyScale\n")
    for row in tbl:
        fp.write("{0},{1},{2},{3},{4},{5},{6}\n".format(*row))